## Imports

In [620]:
from collections import deque 
import numpy as np
import random
import gym
import time as lp

## The input variables

In [600]:
states = 4
actions = 2           

In [601]:
batch_size = 32

In [602]:
episodes = 1000

In [603]:
env = gym.make('CartPole-v0')

In [604]:
env.reset()

array([-0.04395843, -0.03976584,  0.04555846,  0.0064775 ])

## Defining the Agent

In [605]:
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [614]:
state_size = 7 
action_size = 7

In [624]:
class QAgent():
    
    def __init__(self,state_size,action_size):        
        self.state_size = state_size
        self.action_size = action_size        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        
        self.model = self._build_model()
    
    def _build_model(self):
        
        
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size,activation="relu"))          
        model.add(Dense(24,activation = "relu"))
        model.add(Dense(self.action_size,activation = "relu"))
        
        model.compile(loss="mse",optimizer='Adam')  
                  
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self,state,z,a1) :
        min_numb = 50           
        if np.random.rand() <= self.epsilon:
            if all(p<min_numb for p in a1):
                return np.argmax([z/v for v in a1])
            else:
                return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay (self, batch_size):       
        minibatch = random.sample(self.memory,batch_size)       
        for state,action,reward,next_state,done in minibatch : 
            target = reward
            if not done:
                target = (reward + self.gamma*np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target                
            self.model.fit(state,target_f,epochs = 1, verbose = 0)        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
                
    def load(self,name):
        self.model.load_weights(name)
    
    def save(self,name):
        self.model.save_weights(name)
    

def initializer():
    with request.urlopen('https://app.nextel.io/parse/getFbStorePost/920') as response:
            source = response.read()
            data = json.loads(source)


    

    data1 = pd.DataFrame()
    for c in range(len(data)):
        tu = pd.DataFrame(data[c],index=[c])
        data1 = data1.append(tu)

    data1['impression'] = data1["impression"].astype(int)
    data1.reset_index(drop=True)
    
    return data1

def reset(campaign_id):
    
    PPR = 1000 
    
    data1 = initializer()

    ind_ = list(data1[data1['campaign_id']== f'{campaign_id}'].index)[0]

    vnm = data1['adsetsids'][ind_]      
    adset1 = vnm.split('"')
    adset=[]
    for oo in range(len(adset1)):
        if oo%2!=0:
            adset.append(adset1[oo])
    adset
    spend = 0
    budget_dict = {}   
    reset_dict = {}   
    for q in json.loads(data1['adset_breakdown'][ind_])['age'][adset[0]]['data']:
        spend += float(q['spend'])
        chk = []
        try:
            for r in q['cost_per_action_type']:
                chk.append(r['action_type'])
                if r['action_type']== 'post_reaction':  
                    reset_dict[q['age']] = (PPR/(float(r['value'])))*(float(q['spend'])/float(r['value']))
                    budget_dict[q['age']] = float(q['spend'])
            if 'post_reaction' not in chk:
                reset_dict[q['age']] = 0
                budget_dict[q['age']] = float(q['spend'])
        except Exception:
            reset_dict[q['age']] = 0
            budget_dict[q['age']] = float(q['spend'])


    total = sum(reset_dict.values())
    reset_dict1 = {k:(l/total)*spend for k,l in reset_dict.items()}   

    state_after_reset = np.array(list(reset_dict1.values()))
    
    return state_after_reset,reset_dict


def single_action_to_full(reset_dict,action):

    total = sum(reset_dict.values())
    reset_dict_pct = {k:(l/total)*100 for k,l in reset_dict.items()}

    

    range_of_profit_m = [(0,15),(15,25),(25,35),(35,45),(45,55),(55,100)]
    pct_alloted = {'0':1,'15':3,'25':5,'35':6,'45':8,'55':10}

    x = list(reset_dict_pct.values())[action]
    _pct = list(reset_dict_pct.values())[action]      
    for h in range_of_profit_m:
        if int(x) in range(h[0],h[1]):
            increase_pct = (pct_alloted[str(h[0])])
            increased_pct = (increase_pct+ _pct)
            key_abe = (list(reset_dict_pct.keys())[action])        

    reset_dict_del = dict(reset_dict_pct)
    reset_dict_del.pop(key_abe)

    tot = sum(list(reset_dict_del.values()))
    reset_dict_pct_left = {k: (((l/tot)*increase_pct)) for k,l in reset_dict_del.items() }


    key_link = []
    no_zero_dict = {}
    for k,y in reset_dict_pct_left.items():
        if y!=0:
            no_zero_dict[k] = y 
        else:
            key_link.append(k)

        without_zeros = len(no_zero_dict)

    new_pct_dist = {}

    i = 0
    o = 0

    for b in range(without_zeros):
        next_ = max(reset_dict_pct_left.values())
        left = increase_pct-next_
        left_key = max(reset_dict_pct_left,key=reset_dict_pct_left.get)
        if reset_dict_pct[left_key]*0.4>left:
            new_pct_dist[left_key] = left
        else:
            new_left = left-reset_dict_pct[left_key]*0.4
            new_pct_dist[left_key] = reset_dict_pct[left_key]*0.4
            if i==without_zeros:
                o+=new_left
            else:
                next_ += new_left
        reset_dict_pct_left.pop(left_key)
        rest_pct = next_
        i += 1
    if o!=0:
        increased_pct += o


    reset_dict_pct[key_abe] = increased_pct

    new_pct_dist[key_abe]=0
    for b in key_link:
        new_pct_dist[b]=0

    final = {k: ((reset_dict_pct[k]-new_pct_dist[k])/100)*spend for k,i in reset_dict_pct.items()}
    next_state = np.array(list(final.values()))
    
    return next_state

def Step(action,campaign_id):
    one ,two = reset(campaign_id)
    old_sum_PM = sum(list(two.values()))
    next_state = single_action_to_full(two,action)   
    lp.sleep(10)                                            
    one ,two_new = reset(campaign_id)
    new_sum_PM = sum(list(two_new.values()))
    reward = (new_sum_PM-old_sum_PM)*100
    if (abs(new_sum_PM-old_sum_PM)/new_sum_PM)*100 > 25:
        done = True
    else:
        done = False
    return next_state,reward,done

In [625]:
agent = QAgent(state_size = state_size, action_size = action_size)

In [ ]:
done = False 
z = 0
a1 = [1,1,1,1,1,1,1]       
for e in range(episodes):  
    
    campaign_id = 6010678584024
    state,_ = reset(campaign_id)
    state =  state.reshape((1,state_size))  
                                            
                                            
                                   
    
    for time2 in range(2):   
        
        action = agent.act(state,z,a1)
        
        z+=1
        
        a1[action]+=1
        
        next_state, reward, done = Step(action,campaign_id)
        
        reward = reward if not done else -10  
        
        next_state = next_state.reshape((1,state_size))
        
        agent.remember(state,action,reward,next_state,done) 
        
        state = next_state
        
#         if done :
        print(f"episodes : {e}/{episodes} , score : {time2} , epsilon : {agent.epsilon:.2} ")
#         break

    if len(agent.memory) > batch_size:      # Save model 
        agent.replay(batch_size)
        
    
    agent.save(directory+'weights'+'{:04d}'.format(e)+".hdf5")